# 🚀 Microsoft Fabric – Warehouse Performance & OneLake Soft Delete Monitor

This notebook provides two key capabilities for Microsoft Fabric governance:

**Part A – Warehouse Performance Diagnostics** based on [Performance guidelines in Fabric Data Warehouse](https://learn.microsoft.com/en-us/fabric/data-warehouse/guidelines-warehouse-performance):

| Check | Source |
|-------|--------|
| Cold Cache Detection | `queryinsights.exec_requests_history` – `data_scanned_remote_storage_mb` |
| Long-Running Queries | `queryinsights.long_running_queries` |
| Frequently Run Queries | `queryinsights.frequently_run_queries` |
| Statistics Freshness | `sys.stats` + `DBCC SHOW_STATISTICS` |
| String Column Sizing | `sys.columns` – oversized `varchar(8000)` / `varchar(max)` detection |
| Data Type Optimization | `sys.columns` – nullable columns, `decimal(18,0)` overuse |
| Active Locks & Blocking | `sys.dm_tran_locks` |
| Table Compaction Health | `sys.dm_db_partition_stats` – small/fragmented row groups |
| V-Order Status | Fabric REST API – warehouse properties |

**Part B – OneLake Soft Delete Scanner** based on [OneLake Disaster Recovery – Soft Delete](https://learn.microsoft.com/en-us/fabric/onelake/onelake-disaster-recovery#soft-delete-for-onelake-files):

| Check | Source |
|-------|--------|
| Soft-Deleted Files | OneLake DFS API via `azure-storage-file-datalake` |
| Remaining Recovery Days | Blob metadata – `RemainingDaysBeforePermanentDelete` |
| Storage Cost of Deleted Data | Calculated from soft-deleted file sizes |

```mermaid
flowchart LR
    A["🔐 Fabric Auth"] --> B["📡 Fabric REST API"]
    A --> C["🗄️ SQL Endpoint"]
    A --> D["📂 OneLake DFS"]
    B --> E["📦 Discover Items"]
    C --> F["📊 Query Insights"]
    C --> G["🔍 Schema Analysis"]
    D --> H["🗑️ Soft Delete Scan"]
    F --> I["📋 Performance Report"]
    G --> I
    H --> J["📋 Soft Delete Report"]
    I --> K["💾 CSV Export"]
    J --> K
```

## Prerequisites
- This notebook runs **inside Microsoft Fabric** (not externally)
- Fabric workspace Admin or Member role
- Pre-installed libraries: `sempy`, `mssparkutils`, `pandas`, `requests`, `pyspark`
- For Soft Delete scanning: `azure-storage-file-datalake` (install cell provided)

## 0️⃣ Verify Fabric Environment & Install Dependencies

In [ ]:
# ── Verify Fabric Environment ─────────────────────────────────────────

import sempy.fabric as fabric
from notebookutils import mssparkutils
import pandas as pd
import requests
import json
import re
import time
from datetime import datetime, timedelta, timezone

print("✅ Fabric environment verified")
print(f"   sempy version:      {fabric.__version__ if hasattr(fabric, '__version__') else 'OK'}")
print(f"   mssparkutils:       available")
print(f"   pandas version:     {pd.__version__}")
print(f"   spark:              {spark.version}")

# Install azure-storage-file-datalake for soft-delete scanning
try:
    from azure.storage.filedatalake import DataLakeServiceClient
    print("   azure-storage-file-datalake: already installed")
except ImportError:
    print("   Installing azure-storage-file-datalake ...")
    %pip install -q azure-storage-file-datalake
    from azure.storage.filedatalake import DataLakeServiceClient
    print("   azure-storage-file-datalake: installed ✅")

## 1️⃣ Configuration

In [ ]:
# ── Configuration ────────────────────────────────────────────────────

# ── Fabric Workspace ───────────────────────────────────────────────
# Set to None to use the current workspace, or specify a workspace GUID
TARGET_WORKSPACE_ID = None

# ── Feature Toggles ────────────────────────────────────────────────
# Set to True/False to enable/disable each section
RUN_PERFORMANCE_DIAGNOSTICS = True   # Part A: Warehouse performance checks
RUN_SOFT_DELETE_SCAN        = True   # Part B: OneLake soft delete scan

# ── Performance Diagnostics Settings ────────────────────────────────
# Thresholds for performance alerts
COLD_CACHE_THRESHOLD_MB     = 100    # Flag queries reading > N MB from remote storage
LONG_QUERY_THRESHOLD_SEC    = 300    # Flag queries running longer than N seconds
VARCHAR_OVERSIZED_THRESHOLD = 4000   # Flag varchar(n) columns where n > this value
TOP_N_QUERIES               = 20     # Number of top queries to show in reports

# ── Soft Delete Settings ────────────────────────────────────────────
SCAN_ITEM_TYPES = ("Lakehouse", "Warehouse")  # Item types to scan for soft deletes

# ── Resolve workspace ID ────────────────────────────────────────────
if TARGET_WORKSPACE_ID:
    WORKSPACE_ID = TARGET_WORKSPACE_ID
else:
    WORKSPACE_ID = fabric.get_workspace_id()

# ── Fabric API ─────────────────────────────────────────────────────
FABRIC_API_BASE = "https://api.fabric.microsoft.com/v1"

# ── Authentication ─────────────────────────────────────────────────
fabric_token = mssparkutils.credentials.getToken("pbi")
sql_token    = mssparkutils.credentials.getToken("https://database.windows.net")

HEADERS = {
    "Authorization": f"Bearer {fabric_token}",
    "Content-Type": "application/json"
}

# ── Display Options ────────────────────────────────────────────────
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.float_format', '{:,.2f}'.format)

# ── Validate workspace ─────────────────────────────────────────────
resp = requests.get(f"{FABRIC_API_BASE}/workspaces/{WORKSPACE_ID}", headers=HEADERS)
resp.raise_for_status()
ws = resp.json()
print(f"🚀 Warehouse Performance & Soft Delete Monitor")
print(f"   Workspace:      {ws['displayName']}")
print(f"   Workspace ID:   {WORKSPACE_ID}")
print(f"   Capacity:       {ws.get('capacityId', 'N/A')}")
print(f"   Performance:    {'✅ Enabled' if RUN_PERFORMANCE_DIAGNOSTICS else '⏭️ Skipped'}")
print(f"   Soft Delete:    {'✅ Enabled' if RUN_SOFT_DELETE_SCAN else '⏭️ Skipped'}")

## 2️⃣ Discover Warehouses & Lakehouses

Identifies all Warehouses and Lakehouses in the workspace for both performance analysis and soft-delete scanning.

In [ ]:
# ── Discover Warehouses & Lakehouses ────────────────────────────────

def list_fabric_items(workspace_id, item_type, headers):
    """List all items of a given type in a Fabric workspace (handles pagination)."""
    url = f"{FABRIC_API_BASE}/workspaces/{workspace_id}/{item_type}"
    items = []
    while url:
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        data = resp.json()
        items.extend(data.get("value", []))
        url = data.get("continuationUri")
    return items

# ── Warehouses ─────────────────────────────────────────────────────
raw_warehouses = list_fabric_items(WORKSPACE_ID, "warehouses", HEADERS)
warehouses = []
for w in raw_warehouses:
    props = w.get("properties", {})
    conn_string = props.get("connectionString", "")
    sql_ep = conn_string.replace("jdbc:sqlserver://", "").split(";")[0] if conn_string else ""
    warehouses.append({
        "item_id":       w["id"],
        "item_name":     w["displayName"],
        "item_type":     "Warehouse",
        "sql_endpoint":  sql_ep,
        "database_name": w["displayName"],
    })
print(f"📦 Found {len(warehouses)} Warehouse(s)")

# ── Lakehouses ─────────────────────────────────────────────────────
raw_lakehouses = list_fabric_items(WORKSPACE_ID, "lakehouses", HEADERS)
lakehouses = []
for lh in raw_lakehouses:
    props = lh.get("properties", {})
    sql_ep = props.get("sqlEndpointProperties", {}).get("connectionString", "")
    lakehouses.append({
        "item_id":       lh["id"],
        "item_name":     lh["displayName"],
        "item_type":     "Lakehouse",
        "sql_endpoint":  sql_ep,
        "database_name": lh["displayName"],
    })
print(f"🏠 Found {len(lakehouses)} Lakehouse(s)")

# ── Combined list ───────────────────────────────────────────────────
all_items = warehouses + lakehouses
print(f"\n✅ Total: {len(all_items)} item(s)")
for item in all_items:
    print(f"   {item['item_type']:10s} | {item['item_name']} | SQL: {item['sql_endpoint'][:60] if item['sql_endpoint'] else 'N/A'}")

---
# Part A: Warehouse Performance Diagnostics

Performance checks based on [Microsoft Fabric Warehouse Performance Guidelines](https://learn.microsoft.com/en-us/fabric/data-warehouse/guidelines-warehouse-performance).

## 3️⃣ SQL Endpoint Query Helper

Reusable function to query Warehouse/Lakehouse SQL endpoints via Spark JDBC.

In [ ]:
# ── SQL Endpoint Query Helper ────────────────────────────────────────

def query_sql_endpoint(sql_endpoint, database_name, access_token, query):
    """Query a Fabric SQL endpoint using Spark JDBC. Returns a pandas DataFrame."""
    jdbc_url = (
        f"jdbc:sqlserver://{sql_endpoint};"
        f"database={database_name};"
        f"encrypt=true;"
        f"trustServerCertificate=false;"
        f"loginTimeout=30;"
    )
    spark_df = (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", query)
        .option("accessToken", access_token)
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
        .load()
    )
    try:
        return spark_df.toPandas()
    except (TypeError, ValueError):
        from pyspark.sql.functions import col
        for field in spark_df.schema.fields:
            if "date" in field.dataType.simpleString() or "timestamp" in field.dataType.simpleString():
                spark_df = spark_df.withColumn(field.name, col(field.name).cast("string"))
        return spark_df.toPandas()


def safe_query(sql_endpoint, database_name, access_token, query, label="query"):
    """Execute a query with error handling. Returns DataFrame or None."""
    try:
        return query_sql_endpoint(sql_endpoint, database_name, access_token, query)
    except Exception as e:
        print(f"      ⚠️  {label} failed: {str(e)[:120]}")
        return None

## 4️⃣ Cold Cache & Remote Storage Detection

Queries `queryinsights.exec_requests_history` to detect queries that fetched data from remote storage (OneLake) instead of cache.
A non-zero `data_scanned_remote_storage_mb` indicates a **cold start** — the query had to load data from OneLake into memory.

> **Best Practice:** Don't judge query performance based on the first execution. Subsequent runs use cached data and are significantly faster.

In [ ]:
# ── Cold Cache & Remote Storage Detection ────────────────────────────

COLD_CACHE_QUERY = f"""(
SELECT TOP {TOP_N_QUERIES}
    CONVERT(VARCHAR(30), start_time, 126)        AS start_time,
    CONVERT(VARCHAR(30), end_time, 126)          AS end_time,
    DATEDIFF(SECOND, start_time, end_time)       AS duration_sec,
    data_scanned_remote_storage_mb,
    data_scanned_memory_mb,
    CAST(command AS VARCHAR(500))                 AS query_text,
    status,
    session_id,
    distributed_statement_id
FROM queryinsights.exec_requests_history
WHERE data_scanned_remote_storage_mb > {COLD_CACHE_THRESHOLD_MB}
ORDER BY data_scanned_remote_storage_mb DESC
) AS cold_cache"""

if RUN_PERFORMANCE_DIAGNOSTICS:
    all_cold_cache = []

    for item in warehouses:
        sql_ep    = item["sql_endpoint"]
        item_name = item["item_name"]

        if not sql_ep:
            continue

        print(f"❄️  Checking cold cache for: {item_name} ...")
        df = safe_query(sql_ep, item["database_name"], sql_token,
                        COLD_CACHE_QUERY, "Cold cache query")
        if df is not None and not df.empty:
            df["warehouse_name"] = item_name
            all_cold_cache.append(df)
            print(f"   Found {len(df)} queries with significant remote storage reads")
        else:
            print(f"   ✅ No significant cold cache hits detected")

    if all_cold_cache:
        df_cold_cache = pd.concat(all_cold_cache, ignore_index=True)
        print(f"\n❄️  COLD CACHE REPORT")
        print("=" * 120)
        print(f"   Queries fetching > {COLD_CACHE_THRESHOLD_MB} MB from remote storage: {len(df_cold_cache)}")
        print()
        display_cols = ["warehouse_name", "start_time", "duration_sec",
                        "data_scanned_remote_storage_mb", "data_scanned_memory_mb", "query_text"]
        print(df_cold_cache[[c for c in display_cols if c in df_cold_cache.columns]].to_string(index=False))
        print("\n💡 Tip: Subsequent executions of these queries should be faster once data is cached.")
    else:
        df_cold_cache = pd.DataFrame()
        print("\n✅ No significant cold cache issues detected across all warehouses.")
else:
    print("⏭️  Performance diagnostics skipped (RUN_PERFORMANCE_DIAGNOSTICS = False)")

## 5️⃣ Long-Running & Frequently Run Queries

Uses the built-in `queryinsights` views to identify:
- **Long-running queries** – queries that take an unusually long time to execute
- **Frequently run queries** – queries executed most often (candidates for optimization or caching)

In [ ]:
# ── Long-Running & Frequently Run Queries ────────────────────────────

LONG_RUNNING_QUERY = f"""(
SELECT TOP {TOP_N_QUERIES}
    CAST(query_hash AS VARCHAR(100))             AS query_hash,
    CAST(query_text AS VARCHAR(500))             AS query_text,
    execution_count,
    median_duration_ms,
    median_duration_ms / 1000.0                  AS median_duration_sec,
    max_duration_ms,
    max_duration_ms / 1000.0                     AS max_duration_sec,
    median_row_count,
    CONVERT(VARCHAR(30), last_execution_start_time, 126) AS last_execution
FROM queryinsights.long_running_queries
ORDER BY median_duration_ms DESC
) AS long_running"""

FREQUENTLY_RUN_QUERY = f"""(
SELECT TOP {TOP_N_QUERIES}
    CAST(query_hash AS VARCHAR(100))             AS query_hash,
    CAST(query_text AS VARCHAR(500))             AS query_text,
    execution_count,
    median_duration_ms,
    median_duration_ms / 1000.0                  AS median_duration_sec,
    median_row_count,
    CONVERT(VARCHAR(30), last_execution_start_time, 126) AS last_execution
FROM queryinsights.frequently_run_queries
ORDER BY execution_count DESC
) AS freq_run"""

if RUN_PERFORMANCE_DIAGNOSTICS:
    all_long_running = []
    all_frequent = []

    for item in warehouses:
        sql_ep    = item["sql_endpoint"]
        item_name = item["item_name"]

        if not sql_ep:
            continue

        print(f"🐢 Checking long-running queries for: {item_name} ...")
        df_lr = safe_query(sql_ep, item["database_name"], sql_token,
                           LONG_RUNNING_QUERY, "Long-running queries")
        if df_lr is not None and not df_lr.empty:
            df_lr["warehouse_name"] = item_name
            all_long_running.append(df_lr)
            print(f"   Found {len(df_lr)} long-running query pattern(s)")

        print(f"🔄 Checking frequently run queries for: {item_name} ...")
        df_fr = safe_query(sql_ep, item["database_name"], sql_token,
                           FREQUENTLY_RUN_QUERY, "Frequent queries")
        if df_fr is not None and not df_fr.empty:
            df_fr["warehouse_name"] = item_name
            all_frequent.append(df_fr)
            print(f"   Found {len(df_fr)} frequently executed query pattern(s)")

    # ── Long-Running Report ──────────────────────────────────────────
    if all_long_running:
        df_long_running = pd.concat(all_long_running, ignore_index=True)
        print(f"\n🐢 LONG-RUNNING QUERIES")
        print("=" * 130)
        for _, row in df_long_running.iterrows():
            dur = row.get("median_duration_sec", 0) or 0
            icon = "🔴" if dur > LONG_QUERY_THRESHOLD_SEC else "🟡"
            print(f"   {icon} [{row.get('warehouse_name', '')}] "
                  f"Median: {dur:.1f}s | Max: {row.get('max_duration_sec', 0):.1f}s | "
                  f"Runs: {row.get('execution_count', 0)} | "
                  f"{str(row.get('query_text', ''))[:80]}")
    else:
        df_long_running = pd.DataFrame()
        print("\n✅ No long-running queries detected.")

    # ── Frequently Run Report ─────────────────────────────────────────
    if all_frequent:
        df_frequent = pd.concat(all_frequent, ignore_index=True)
        print(f"\n🔄 FREQUENTLY RUN QUERIES")
        print("=" * 130)
        for _, row in df_frequent.iterrows():
            print(f"   📊 [{row.get('warehouse_name', '')}] "
                  f"Executions: {row.get('execution_count', 0):,} | "
                  f"Median: {row.get('median_duration_sec', 0):.1f}s | "
                  f"{str(row.get('query_text', ''))[:80]}")
        print(f"\n💡 Tip: Frequently run queries are prime candidates for optimization and caching.")
    else:
        df_frequent = pd.DataFrame()

## 6️⃣ Statistics Freshness Check

Checks whether **auto-created statistics** are stale or missing. Fabric Warehouse automatically maintains histogram statistics, average column length statistics, and table cardinality statistics. However, if there's a large window between table transformations and query workloads, manually updating statistics during a maintenance window can improve query performance.

> **Best Practice:** Use `CREATE STATISTICS` or `UPDATE STATISTICS` T-SQL commands during maintenance windows to reduce the likelihood of `SELECT` queries having to first update statistics.

In [ ]:
# ── Statistics Freshness Check ───────────────────────────────────────

STATS_QUERY = """(
SELECT
    s.name                                       AS schema_name,
    t.name                                       AS table_name,
    st.name                                      AS stats_name,
    st.auto_created,
    st.user_created,
    CONVERT(VARCHAR(30), sp.last_updated, 126)   AS last_updated,
    sp.rows,
    sp.rows_sampled,
    sp.modification_counter
FROM sys.stats st
INNER JOIN sys.tables t
    ON st.object_id = t.object_id
INNER JOIN sys.schemas s
    ON t.schema_id = s.schema_id
CROSS APPLY sys.dm_db_stats_properties(st.object_id, st.stats_id) sp
WHERE t.is_ms_shipped = 0
ORDER BY sp.modification_counter DESC, sp.last_updated ASC
) AS stats_info"""

if RUN_PERFORMANCE_DIAGNOSTICS:
    all_stats = []

    for item in warehouses:
        sql_ep = item["sql_endpoint"]
        item_name = item["item_name"]

        if not sql_ep:
            continue

        print(f"📊 Checking statistics freshness for: {item_name} ...")
        df = safe_query(sql_ep, item["database_name"], sql_token,
                        STATS_QUERY, "Statistics freshness")
        if df is not None and not df.empty:
            df["warehouse_name"] = item_name
            all_stats.append(df)

            # Flag stale stats (high modification counter relative to rows)
            stale = df[df["modification_counter"] > 0].copy()
            if not stale.empty:
                stale["stale_pct"] = (stale["modification_counter"] / stale["rows"].clip(lower=1) * 100)
                very_stale = stale[stale["stale_pct"] > 20]
                if not very_stale.empty:
                    print(f"   ⚠️  {len(very_stale)} statistics are >20% stale (rows modified since last stats update)")
                    for _, row in very_stale.head(5).iterrows():
                        print(f"      - {row['schema_name']}.{row['table_name']}.{row['stats_name']}: "
                              f"{row['modification_counter']:,} modifications ({row['stale_pct']:.0f}% of rows)")
                else:
                    print(f"   ✅ All statistics are reasonably fresh")
            else:
                print(f"   ✅ No modified rows since last statistics update")

    if all_stats:
        df_all_stats = pd.concat(all_stats, ignore_index=True)
        print(f"\n📊 STATISTICS FRESHNESS SUMMARY")
        print("=" * 100)
        total_stats = len(df_all_stats)
        stale_stats = len(df_all_stats[df_all_stats["modification_counter"] > 0])
        print(f"   Total statistics objects: {total_stats}")
        print(f"   With modifications:       {stale_stats}")
        print(f"\n💡 Tip: Run 'UPDATE STATISTICS <table>' during maintenance windows for stale statistics.")
    else:
        df_all_stats = pd.DataFrame()

## 7️⃣ Schema & Data Type Analysis

Identifies columns that may hurt performance based on the warehouse performance guidelines:

| Issue | Impact | Recommendation |
|-------|--------|----------------|
| `varchar(8000)` / `varchar(max)` | Inaccurate statistics, suboptimal query plans | Use specific `varchar(n)` lengths |
| Oversized `decimal(18,0)` | Wastes 9 bytes/row when `int` (4 bytes) suffices | Use `int`/`bigint` for whole numbers |
| Nullable columns | Metadata overhead, reduced optimization | Use `NOT NULL` when possible |
| `char(n)` for variable-length data | Wasted space, I/O overhead | Use `varchar(n)` unless fixed-length |
| Data type mismatches in JOINs | Implicit conversions, poor performance | Ensure JOIN columns have matching types |

In [ ]:
# ── Schema & Data Type Analysis ──────────────────────────────────────

SCHEMA_ANALYSIS_QUERY = f"""(
SELECT
    s.name                  AS schema_name,
    t.name                  AS table_name,
    c.name                  AS column_name,
    ty.name                 AS data_type,
    c.max_length,
    c.precision,
    c.scale,
    c.is_nullable,
    c.column_id
FROM sys.columns c
INNER JOIN sys.tables t  ON c.object_id = t.object_id
INNER JOIN sys.schemas s ON t.schema_id = s.schema_id
INNER JOIN sys.types ty  ON c.user_type_id = ty.user_type_id
WHERE t.is_ms_shipped = 0
ORDER BY s.name, t.name, c.column_id
) AS schema_info"""

if RUN_PERFORMANCE_DIAGNOSTICS:
    all_schema_issues = []
    all_schema_data = []

    for item in warehouses + lakehouses:
        sql_ep = item["sql_endpoint"]
        item_name = item["item_name"]
        item_type = item["item_type"]

        if not sql_ep:
            continue

        print(f"🔍 Analyzing schema for {item_type}: {item_name} ...")
        df = safe_query(sql_ep, item["database_name"], sql_token,
                        SCHEMA_ANALYSIS_QUERY, "Schema analysis")
        if df is None or df.empty:
            continue

        df["item_name"] = item_name
        df["item_type"] = item_type
        all_schema_data.append(df)

        issues = []

        # Check 1: Oversized varchar columns
        oversized_varchar = df[
            (df["data_type"].isin(["varchar", "nvarchar"])) &
            (df["max_length"] > VARCHAR_OVERSIZED_THRESHOLD)
        ]
        if not oversized_varchar.empty:
            for _, row in oversized_varchar.iterrows():
                length_display = "max" if row["max_length"] == -1 else str(row["max_length"])
                issues.append({
                    "item_name": item_name,
                    "item_type": item_type,
                    "table": f"{row['schema_name']}.{row['table_name']}",
                    "column": row["column_name"],
                    "issue": f"Oversized string: {row['data_type']}({length_display})",
                    "severity": "🔴 High",
                    "recommendation": f"Use {row['data_type']}(n) with appropriate max length"
                })

        # Check 2: decimal(18,0) – could often be int/bigint
        inefficient_decimal = df[
            (df["data_type"].isin(["decimal", "numeric"])) &
            (df["precision"] == 18) & (df["scale"] == 0)
        ]
        if not inefficient_decimal.empty:
            for _, row in inefficient_decimal.iterrows():
                issues.append({
                    "item_name": item_name,
                    "item_type": item_type,
                    "table": f"{row['schema_name']}.{row['table_name']}",
                    "column": row["column_name"],
                    "issue": "decimal(18,0) – 9 bytes/row",
                    "severity": "🟡 Medium",
                    "recommendation": "Consider int (4 bytes) or bigint (8 bytes) for whole numbers"
                })

        # Check 3: Nullable columns that could be NOT NULL
        nullable_cols = df[df["is_nullable"] == True]
        nullable_pct = len(nullable_cols) / len(df) * 100 if len(df) > 0 else 0
        if nullable_pct > 80:
            issues.append({
                "item_name": item_name,
                "item_type": item_type,
                "table": "(all tables)",
                "column": "(multiple)",
                "issue": f"{nullable_pct:.0f}% of columns are nullable",
                "severity": "🟡 Medium",
                "recommendation": "Define columns as NOT NULL when the data model allows"
            })

        # Check 4: char vs varchar opportunity
        char_cols = df[df["data_type"].isin(["char", "nchar"])]
        large_char = char_cols[char_cols["max_length"] > 20]
        if not large_char.empty:
            for _, row in large_char.iterrows():
                issues.append({
                    "item_name": item_name,
                    "item_type": item_type,
                    "table": f"{row['schema_name']}.{row['table_name']}",
                    "column": row["column_name"],
                    "issue": f"Fixed-length char({row['max_length']})",
                    "severity": "🟡 Medium",
                    "recommendation": "Use varchar(n) unless data is truly fixed-length"
                })

        all_schema_issues.extend(issues)

        if issues:
            print(f"   ⚠️  {len(issues)} schema optimization issue(s) found")
        else:
            print(f"   ✅ Schema looks well-optimized")

    # ── Schema Issues Report ─────────────────────────────────────────
    if all_schema_issues:
        df_schema_issues = pd.DataFrame(all_schema_issues)
        print(f"\n🔍 SCHEMA & DATA TYPE OPTIMIZATION REPORT")
        print("=" * 130)
        print(f"   Total issues found: {len(df_schema_issues)}")

        high = len(df_schema_issues[df_schema_issues["severity"].str.contains("High")])
        medium = len(df_schema_issues[df_schema_issues["severity"].str.contains("Medium")])
        print(f"   🔴 High:   {high}")
        print(f"   🟡 Medium: {medium}")
        print()

        for _, row in df_schema_issues.iterrows():
            print(f"   {row['severity']} [{row['item_name']}] {row['table']}.{row['column']}")
            print(f"      Issue: {row['issue']}")
            print(f"      Fix:   {row['recommendation']}")
            print()
    else:
        df_schema_issues = pd.DataFrame()
        print("\n✅ No schema optimization issues detected.")

## 8️⃣ Active Locks & Transaction Monitoring

Checks for active locks and potential blocking using `sys.dm_tran_locks`. Long-running explicit transactions can cause contention with `SELECT` statements on system catalog views and the Fabric portal.

> **Best Practice:** Keep transactions short-lived. Always `COMMIT` or `ROLLBACK`. Add retry logic with exponential backoff in pipelines.

In [ ]:
# ── Active Locks & Transaction Monitoring ────────────────────────────

LOCKS_QUERY = """(
SELECT
    l.resource_type,
    l.request_mode,
    l.request_status,
    l.request_session_id     AS session_id,
    CAST(t.name AS VARCHAR(128))   AS table_name,
    COUNT(*)                 AS lock_count
FROM sys.dm_tran_locks l
LEFT JOIN sys.tables t ON l.resource_associated_entity_id = t.object_id
    AND l.resource_type = 'OBJECT'
GROUP BY l.resource_type, l.request_mode, l.request_status,
         l.request_session_id, CAST(t.name AS VARCHAR(128))
HAVING COUNT(*) > 0
) AS lock_info"""

if RUN_PERFORMANCE_DIAGNOSTICS:
    all_locks = []

    for item in warehouses:
        sql_ep = item["sql_endpoint"]
        item_name = item["item_name"]

        if not sql_ep:
            continue

        print(f"🔒 Checking active locks for: {item_name} ...")
        df = safe_query(sql_ep, item["database_name"], sql_token,
                        LOCKS_QUERY, "Lock inspection")
        if df is not None and not df.empty:
            df["warehouse_name"] = item_name
            all_locks.append(df)

            # Check for blocked requests (WAIT status)
            blocked = df[df["request_status"] == "WAIT"]
            if not blocked.empty:
                print(f"   ⚠️  {len(blocked)} blocked lock request(s) detected!")
            else:
                print(f"   ✅ Active locks present but no blocking ({len(df)} lock entries)")
        else:
            print(f"   ✅ No active locks")

    if all_locks:
        df_locks = pd.concat(all_locks, ignore_index=True)
        blocked_total = len(df_locks[df_locks["request_status"] == "WAIT"])
        if blocked_total > 0:
            print(f"\n🔒 LOCK CONTENTION ALERT")
            print("=" * 80)
            print(f"   Blocked requests: {blocked_total}")
            print(f"\n💡 Tip: Investigate long-running transactions. Keep transactions short and use retry logic.")
    else:
        df_locks = pd.DataFrame()

## 9️⃣ Table Compaction & Row Group Health

Checks for fragmented or suboptimal row groups that indicate tables needing compaction:
- **Warehouse:** Data compaction runs automatically in the background
- **Lakehouse:** Requires manual `OPTIMIZE` command via Spark or the Fabric portal

> **Best Practice:** Avoid trickle inserts/updates/deletes. Batch operations to create efficient row groups (≥100 MB files). For Lakehouse, run `OPTIMIZE` after significant data changes.

In [ ]:
# ── Table Compaction & Row Group Health ──────────────────────────────

ROWGROUP_QUERY = """(
SELECT
    s.name                                       AS schema_name,
    t.name                                       AS table_name,
    COUNT(DISTINCT ps.partition_number)           AS partition_count,
    SUM(ps.row_count)                            AS total_rows,
    COUNT(ps.partition_number)                   AS total_row_groups,
    SUM(CASE
        WHEN ps.row_count < 102400 THEN 1
        ELSE 0
    END)                                         AS small_row_groups,
    AVG(ps.row_count)                            AS avg_rows_per_group,
    MIN(ps.row_count)                            AS min_rows_in_group,
    MAX(ps.row_count)                            AS max_rows_in_group
FROM sys.dm_db_partition_stats ps
INNER JOIN sys.tables t  ON ps.object_id = t.object_id
INNER JOIN sys.schemas s ON t.schema_id = s.schema_id
WHERE t.is_ms_shipped = 0
    AND ps.index_id IN (0, 1)
GROUP BY s.name, t.name
HAVING SUM(ps.row_count) > 0
ORDER BY SUM(CASE WHEN ps.row_count < 102400 THEN 1 ELSE 0 END) DESC
) AS rowgroup_health"""

if RUN_PERFORMANCE_DIAGNOSTICS:
    all_compaction = []

    for item in warehouses + lakehouses:
        sql_ep = item["sql_endpoint"]
        item_name = item["item_name"]
        item_type = item["item_type"]

        if not sql_ep:
            continue

        print(f"🗜️  Checking row group health for {item_type}: {item_name} ...")
        df = safe_query(sql_ep, item["database_name"], sql_token,
                        ROWGROUP_QUERY, "Row group health")
        if df is not None and not df.empty:
            df["item_name"] = item_name
            df["item_type"] = item_type
            all_compaction.append(df)

            # Flag tables with many small row groups
            fragmented = df[df["small_row_groups"] > 0]
            if not fragmented.empty:
                print(f"   ⚠️  {len(fragmented)} table(s) have small row groups (< 102,400 rows)")
                for _, row in fragmented.head(5).iterrows():
                    pct = (row["small_row_groups"] / max(row["total_row_groups"], 1)) * 100
                    print(f"      - {row['schema_name']}.{row['table_name']}: "
                          f"{row['small_row_groups']}/{row['total_row_groups']} small groups ({pct:.0f}%)")
            else:
                print(f"   ✅ All row groups are healthy")

    if all_compaction:
        df_compaction = pd.concat(all_compaction, ignore_index=True)
        print(f"\n🗜️  TABLE COMPACTION HEALTH")
        print("=" * 120)
        total_tables = len(df_compaction)
        fragmented_tables = len(df_compaction[df_compaction["small_row_groups"] > 0])
        print(f"   Total tables analyzed:      {total_tables}")
        print(f"   Tables with fragmentation:  {fragmented_tables}")

        if fragmented_tables > 0:
            # Separate recommendations for Warehouse vs Lakehouse
            wh_frag = df_compaction[(df_compaction["item_type"] == "Warehouse") & (df_compaction["small_row_groups"] > 0)]
            lh_frag = df_compaction[(df_compaction["item_type"] == "Lakehouse") & (df_compaction["small_row_groups"] > 0)]
            if not wh_frag.empty:
                print(f"\n   📦 Warehouse ({len(wh_frag)} fragmented tables):")
                print(f"      ℹ️  Data compaction runs automatically. Consider batching writes to avoid trickle inserts.")
            if not lh_frag.empty:
                print(f"\n   🏠 Lakehouse ({len(lh_frag)} fragmented tables):")
                print(f"      💡 Run OPTIMIZE in Spark or via Fabric portal → right-click table → Maintenance → Run now")
    else:
        df_compaction = pd.DataFrame()

## 🔟 V-Order & Warehouse Configuration Check

Checks V-Order status for each Warehouse. V-Order is a write-time optimization that improves read performance through better sorting and compression.

| Scenario | Recommendation |
|----------|---------------|
| Read-heavy warehouse | Keep V-Order **ON** (default) |
| Write-heavy staging warehouse | Consider disabling V-Order for ingestion throughput |
| Staging → Serving pattern | V-Order OFF on staging, ON on serving warehouse |

> **Warning:** Once V-Order is disabled on a warehouse, it **cannot be re-enabled**.

In [ ]:
# ── V-Order & Warehouse Configuration ────────────────────────────────

if RUN_PERFORMANCE_DIAGNOSTICS:
    vorder_results = []

    for item in warehouses:
        item_id   = item["item_id"]
        item_name = item["item_name"]

        print(f"⚙️  Checking V-Order for: {item_name} ...")
        try:
            url = f"{FABRIC_API_BASE}/workspaces/{WORKSPACE_ID}/warehouses/{item_id}"
            resp = requests.get(url, headers=HEADERS)
            resp.raise_for_status()
            data = resp.json()
            props = data.get("properties", {})

            # V-Order is ON by default for warehouses
            # The API may not expose it explicitly – it's always ON unless disabled
            vorder_status = "ON (Default)"
            if "vOrderEnabled" in props:
                vorder_status = "ON" if props["vOrderEnabled"] else "OFF ⚠️"

            vorder_results.append({
                "item_name": item_name,
                "item_type": "Warehouse",
                "v_order": vorder_status,
                "connection_string": props.get("connectionString", "N/A")[:80],
            })
            print(f"   V-Order: {vorder_status}")

        except Exception as e:
            print(f"   ⚠️  Could not check: {str(e)[:80]}")

    # Also check Lakehouse maintenance settings
    for item in lakehouses:
        item_id   = item["item_id"]
        item_name = item["item_name"]

        print(f"⚙️  Checking maintenance for Lakehouse: {item_name} ...")
        try:
            url = f"{FABRIC_API_BASE}/workspaces/{WORKSPACE_ID}/lakehouses/{item_id}"
            resp = requests.get(url, headers=HEADERS)
            resp.raise_for_status()
            data = resp.json()
            props = data.get("properties", {})
            maint = props.get("defaultTableMaintenanceConfiguration", {})

            vorder_status = "OFF"
            compaction = "OFF"
            if maint:
                vorder_cfg = maint.get("vOrderOptimization", {})
                vorder_status = "ON" if vorder_cfg.get("enabled", False) else "OFF"
                compact_cfg = maint.get("fileCompaction", {})
                compaction = "ON" if compact_cfg.get("enabled", False) else "OFF"

            vorder_results.append({
                "item_name": item_name,
                "item_type": "Lakehouse",
                "v_order": vorder_status,
                "compaction": compaction,
            })
            print(f"   V-Order: {vorder_status} | Compaction: {compaction}")

        except Exception as e:
            print(f"   ⚠️  Could not check: {str(e)[:80]}")

    if vorder_results:
        df_vorder = pd.DataFrame(vorder_results)
        print(f"\n⚙️  V-ORDER & MAINTENANCE STATUS")
        print("=" * 80)
        print(df_vorder.to_string(index=False))
    else:
        df_vorder = pd.DataFrame()

---
# Part B: OneLake Soft Delete Scanner

Scans OneLake items for **soft-deleted files**. OneLake automatically retains deleted files for **7 days** before permanent removal. This section identifies soft-deleted files you can still recover.

> **Key facts:**
> - Soft-deleted files are retained for 7 days before permanent removal
> - You pay for soft-deleted data at the same rate as active data
> - Write access is required to restore files
> - Use Azure Storage Explorer, PowerShell, or REST APIs to restore

## 1️⃣1️⃣ Scan OneLake for Soft-Deleted Files

Uses the Azure Data Lake Storage SDK to enumerate soft-deleted files across all Lakehouses and Warehouses in the workspace.

In [ ]:
# ── Scan OneLake for Soft-Deleted Files ──────────────────────────────

from azure.storage.filedatalake import DataLakeServiceClient
from azure.core.credentials import TokenCredential, AccessToken

class FabricStorageTokenCredential(TokenCredential):
    """Bridge Fabric notebook token to azure-storage credential."""
    def get_token(self, *scopes, **kwargs):
        tok = mssparkutils.credentials.getToken("storage")
        token_value = tok["accessToken"] if isinstance(tok, dict) else tok
        return AccessToken(token_value, int(time.time()) + 3600)


def scan_soft_deleted_files(workspace_id, items, onelake_endpoint="https://onelake.dfs.fabric.microsoft.com"):
    """
    Scan OneLake items for soft-deleted files.
    Returns a list of dicts with deleted file details.
    """
    cred = FabricStorageTokenCredential()
    service = DataLakeServiceClient(account_url=onelake_endpoint, credential=cred)
    fs = service.get_file_system_client(workspace_id)

    all_deleted = []
    total_scanned = 0

    for idx, item in enumerate(items, 1):
        item_id   = item["item_id"]
        item_name = item.get("item_name", item_id)
        item_type = item.get("item_type", "Unknown")

        print(f"[{idx}/{len(items)}] Scanning {item_type}: {item_name} ...")

        try:
            file_count = 0
            deleted_count = 0

            for path_props in fs.get_paths(path=item_id, recursive=True):
                file_count += 1
                total_scanned += 1

                # Check if the file is soft-deleted
                is_deleted = getattr(path_props, "is_deleted", False) or \
                             getattr(path_props, "deleted", False)

                if is_deleted:
                    deleted_count += 1
                    name = getattr(path_props, "name", "") or ""
                    size = int(getattr(path_props, "content_length", 0) or 0)
                    deleted_time = getattr(path_props, "deleted_time", None) or \
                                   getattr(path_props, "deletion_id", None)
                    remaining_days = getattr(path_props, "remaining_retention_days", None)

                    # Get relative path
                    rel_path = name[len(item_id):].lstrip("/") if name.startswith(item_id) else name

                    all_deleted.append({
                        "item_name":      item_name,
                        "item_type":      item_type,
                        "item_id":        item_id,
                        "file_path":      rel_path,
                        "size_bytes":     size,
                        "size_mb":        size / (1024 * 1024),
                        "deleted_time":   str(deleted_time) if deleted_time else "Unknown",
                        "remaining_days": remaining_days,
                    })

            if deleted_count > 0:
                print(f"   🗑️  Found {deleted_count} soft-deleted file(s) (out of {file_count} total)")
            else:
                print(f"   ✅ No soft-deleted files ({file_count} files scanned)")

        except Exception as e:
            print(f"   ⚠️  Error scanning: {str(e)[:100]}")

    return all_deleted, total_scanned


if RUN_SOFT_DELETE_SCAN:
    # Filter items to requested types
    scan_items = [item for item in all_items if item["item_type"] in SCAN_ITEM_TYPES]

    if scan_items:
        print(f"🗑️  Scanning {len(scan_items)} item(s) for soft-deleted files ...\n")
        deleted_files, total_files_scanned = scan_soft_deleted_files(WORKSPACE_ID, scan_items)

        df_deleted = pd.DataFrame(deleted_files) if deleted_files else pd.DataFrame()
        print(f"\n📊 Soft Delete Scan Complete")
        print(f"   Items scanned:        {len(scan_items)}")
        print(f"   Files scanned:        {total_files_scanned:,}")
        print(f"   Soft-deleted files:   {len(deleted_files)}")
    else:
        print("⚠️  No items found matching the requested types for soft-delete scanning.")
        df_deleted = pd.DataFrame()
else:
    print("⏭️  Soft delete scan skipped (RUN_SOFT_DELETE_SCAN = False)")
    df_deleted = pd.DataFrame()

## 1️⃣2️⃣ Soft Delete Report & Recovery Guidance

Detailed report of soft-deleted files with recovery guidance and storage cost impact.

In [ ]:
# ── Soft Delete Report ───────────────────────────────────────────────

def format_bytes(n):
    """Format bytes to human-readable size."""
    units = ["B", "KB", "MB", "GB", "TB"]
    x = float(n)
    i = 0
    while x >= 1024 and i < len(units) - 1:
        x /= 1024
        i += 1
    return f"{x:,.2f} {units[i]}" if i >= 2 else f"{x:,.0f} {units[i]}"


if RUN_SOFT_DELETE_SCAN and not df_deleted.empty:
    print("🗑️  ONELAKE SOFT DELETE REPORT")
    print("=" * 120)
    print(f"   Workspace:          {ws['displayName']}")
    print(f"   Total deleted files: {len(df_deleted)}")

    total_deleted_bytes = df_deleted["size_bytes"].sum()
    print(f"   Total deleted size:  {format_bytes(total_deleted_bytes)}")
    print(f"   ⚠️  You are billed for soft-deleted data at the same rate as active data")
    print("=" * 120)

    # ── Summary by item ──────────────────────────────────────────────
    print("\n📊 Soft-Deleted Files by Item")
    print("-" * 80)
    item_summary = df_deleted.groupby(["item_type", "item_name"]).agg(
        file_count=("file_path", "count"),
        total_size_bytes=("size_bytes", "sum"),
    ).reset_index()
    item_summary["total_size"] = item_summary["total_size_bytes"].apply(format_bytes)

    for _, row in item_summary.iterrows():
        icon = "📦" if row["item_type"] == "Warehouse" else "🏠"
        print(f"   {icon} {row['item_name']}: {row['file_count']} files ({row['total_size']})")

    # ── File extension breakdown ─────────────────────────────────────
    print("\n📁 Deleted Files by Type")
    print("-" * 60)
    df_deleted["file_ext"] = df_deleted["file_path"].apply(
        lambda x: x.rsplit(".", 1)[-1].lower() if "." in str(x) else "unknown"
    )
    ext_summary = df_deleted.groupby("file_ext").agg(
        count=("file_path", "count"),
        size_bytes=("size_bytes", "sum")
    ).sort_values("size_bytes", ascending=False)

    for ext, row in ext_summary.iterrows():
        print(f"   .{ext:12s}  {row['count']:6,} files  {format_bytes(row['size_bytes']):>12s}")

    # ── Urgency – files expiring soon ────────────────────────────────
    if "remaining_days" in df_deleted.columns:
        expiring = df_deleted[df_deleted["remaining_days"].notna()].copy()
        if not expiring.empty:
            expiring["remaining_days"] = pd.to_numeric(expiring["remaining_days"], errors="coerce")
            urgent = expiring[expiring["remaining_days"] <= 2]
            if not urgent.empty:
                print(f"\n🚨 URGENT: {len(urgent)} file(s) expiring within 2 days!")
                print("-" * 80)
                for _, row in urgent.head(10).iterrows():
                    print(f"   ⏰ {row['item_name']}/{row['file_path']} "
                          f"({format_bytes(row['size_bytes'])}) – {row['remaining_days']:.0f} day(s) left")

    # ── Recovery instructions ────────────────────────────────────────
    print(f"\n\n💡 HOW TO RECOVER SOFT-DELETED FILES")
    print("=" * 80)
    print("""
   Option 1: Azure Storage Explorer
   ─────────────────────────────────
   1. Open Azure Storage Explorer → Connect to OneLake
   2. Navigate to the item containing deleted files
   3. Change dropdown to "Active and soft deleted blobs"
   4. Right-click the deleted file → Undelete

   Option 2: PowerShell
   ─────────────────────
   $ctx = New-AzStorageContext -StorageAccountName "onelake" \\
          -UseConnectedAccount -endpoint "fabric.microsoft.com"
   Get-AzStorageBlob -Container "<workspace-name>" -Context $ctx \\
          -Prefix "<item>.Lakehouse/Files/" -IncludeDeleted |
       Where-Object { $_.IsDeleted } | Restore-AzStorageBlob

   Option 3: REST API
   ───────────────────
   PUT https://onelake.blob.fabric.microsoft.com/<workspace>/<path>?comp=undelete
    """)

    # ── Detailed file listing ────────────────────────────────────────
    print(f"\n📋 Detailed Soft-Deleted Files (showing first 50)")
    print("-" * 120)
    display_cols = ["item_type", "item_name", "file_path", "size_mb", "deleted_time", "remaining_days"]
    avail_cols = [c for c in display_cols if c in df_deleted.columns]
    print(df_deleted[avail_cols].head(50).to_string(index=False))

elif RUN_SOFT_DELETE_SCAN:
    print("✅ No soft-deleted files found across all scanned items.")
    print("   OneLake soft delete retains deleted files for 7 days before permanent deletion.")

---
## 1️⃣3️⃣ Consolidated Performance Summary & Recommendations

Combined performance health dashboard with prioritized recommendations.

In [ ]:
# ── Consolidated Performance Summary ─────────────────────────────────

print("📋 CONSOLIDATED PERFORMANCE & DATA PROTECTION SUMMARY")
print("=" * 120)
print(f"   Workspace: {ws['displayName']}")
print(f"   Warehouses: {len(warehouses)}  |  Lakehouses: {len(lakehouses)}")
print(f"   Report Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 120)

recommendations = []
priority_order = {"🔴 Critical": 0, "🟠 High": 1, "🟡 Medium": 2, "🟢 Info": 3}

# ── Part A Summary ───────────────────────────────────────────────
if RUN_PERFORMANCE_DIAGNOSTICS:
    print("\n🚀 PART A: WAREHOUSE PERFORMANCE")
    print("-" * 80)

    # Cold cache
    if 'df_cold_cache' in dir() and not df_cold_cache.empty:
        print(f"   ❄️  Cold Cache:     {len(df_cold_cache)} queries with significant remote storage reads")
        recommendations.append({
            "priority": "🟡 Medium",
            "area": "Cold Cache",
            "recommendation": f"{len(df_cold_cache)} queries fetched data from remote storage. "
                             "Don't judge performance on first execution. Consider warming cache for critical queries."
        })
    else:
        print(f"   ❄️  Cold Cache:     ✅ No issues")

    # Long-running queries
    if 'df_long_running' in dir() and not df_long_running.empty:
        print(f"   🐢 Long-Running:   {len(df_long_running)} slow query patterns")
        recommendations.append({
            "priority": "🟠 High",
            "area": "Query Performance",
            "recommendation": f"{len(df_long_running)} long-running query patterns found. "
                             "Review query plans, add data clustering, and optimize JOINs/filters."
        })
    else:
        print(f"   🐢 Long-Running:   ✅ No issues")

    # Statistics
    if 'df_all_stats' in dir() and not df_all_stats.empty:
        stale = len(df_all_stats[df_all_stats["modification_counter"] > 0])
        print(f"   📊 Statistics:     {stale} with modifications since last update")
        if stale > 0:
            recommendations.append({
                "priority": "🟡 Medium",
                "area": "Statistics",
                "recommendation": f"{stale} statistics have pending modifications. "
                                 "Run UPDATE STATISTICS during maintenance windows."
            })
    else:
        print(f"   📊 Statistics:     ✅ All fresh")

    # Schema issues
    if 'df_schema_issues' in dir() and not df_schema_issues.empty:
        print(f"   🔍 Schema Issues:  {len(df_schema_issues)} optimization opportunities")
        high_issues = len(df_schema_issues[df_schema_issues["severity"].str.contains("High")])
        if high_issues > 0:
            recommendations.append({
                "priority": "🟠 High",
                "area": "Data Types",
                "recommendation": f"{high_issues} columns with oversized string types (varchar(8000)/max). "
                                 "Use specific varchar(n) lengths for better statistics and query plans."
            })
    else:
        print(f"   🔍 Schema Issues:  ✅ Well-optimized")

    # Compaction
    if 'df_compaction' in dir() and not df_compaction.empty:
        fragmented = len(df_compaction[df_compaction["small_row_groups"] > 0])
        print(f"   🗜️  Compaction:     {fragmented} tables with fragmentation")
        if fragmented > 0:
            recommendations.append({
                "priority": "🟡 Medium",
                "area": "Compaction",
                "recommendation": f"{fragmented} tables have small row groups. "
                                 "For Lakehouse: run OPTIMIZE. For Warehouse: batch writes to avoid trickle inserts."
            })
    else:
        print(f"   🗜️  Compaction:     ✅ Healthy")

    # Locks
    if 'df_locks' in dir() and not df_locks.empty:
        blocked = len(df_locks[df_locks["request_status"] == "WAIT"])
        if blocked > 0:
            print(f"   🔒 Locks:          ⚠️  {blocked} blocked request(s)")
            recommendations.append({
                "priority": "🔴 Critical",
                "area": "Locking",
                "recommendation": f"{blocked} blocked lock requests detected. "
                                 "Investigate long-running transactions. Keep transactions short-lived."
            })
        else:
            print(f"   🔒 Locks:          ✅ No blocking")
    else:
        print(f"   🔒 Locks:          ✅ No active locks")

# ── Part B Summary ───────────────────────────────────────────────
if RUN_SOFT_DELETE_SCAN:
    print(f"\n🗑️  PART B: ONELAKE SOFT DELETE")
    print("-" * 80)

    if not df_deleted.empty:
        total_deleted_size = df_deleted["size_bytes"].sum()
        print(f"   Soft-deleted files:  {len(df_deleted)}")
        print(f"   Deleted data size:   {format_bytes(total_deleted_size)}")
        print(f"   💰 Storage cost:     Same as active data – review if unneeded")

        if total_deleted_size > 1073741824:  # > 1 GB
            recommendations.append({
                "priority": "🟠 High",
                "area": "Soft Delete",
                "recommendation": f"{format_bytes(total_deleted_size)} of soft-deleted data. "
                                 "You are billed at the same rate as active data. Review if this data needs recovery."
            })
        else:
            recommendations.append({
                "priority": "🟢 Info",
                "area": "Soft Delete",
                "recommendation": f"{len(df_deleted)} soft-deleted files ({format_bytes(total_deleted_size)}). "
                                 "Files auto-expire after 7 days. Recover using Storage Explorer or PowerShell if needed."
            })
    else:
        print(f"   ✅ No soft-deleted files found")

# ── Prioritized Recommendations ──────────────────────────────────
print(f"\n\n💡 PRIORITIZED RECOMMENDATIONS")
print("=" * 120)

if recommendations:
    sorted_recs = sorted(recommendations, key=lambda x: priority_order.get(x["priority"], 99))
    for i, rec in enumerate(sorted_recs, 1):
        print(f"\n   {i}. {rec['priority']} [{rec['area']}]")
        print(f"      {rec['recommendation']}")
else:
    print("   ✅ All checks passed! Your workspace is performing well.")

print(f"\n\n📚 REFERENCE LINKS")
print("-" * 80)
print("   📖 Performance Guidelines:  https://learn.microsoft.com/en-us/fabric/data-warehouse/guidelines-warehouse-performance")
print("   📖 OneLake Soft Delete:      https://learn.microsoft.com/en-us/fabric/onelake/onelake-disaster-recovery#soft-delete-for-onelake-files")
print("   📖 Recover Deleted Files:    https://learn.microsoft.com/en-us/fabric/onelake/soft-delete")
print("   📖 Query Insights:           https://learn.microsoft.com/en-us/fabric/data-warehouse/query-insights")
print("   📖 Data Clustering:          https://learn.microsoft.com/en-us/fabric/data-warehouse/data-clustering")
print("   📖 Table Maintenance:        https://learn.microsoft.com/en-us/fabric/data-engineering/lakehouse-table-maintenance")

## 1️⃣4️⃣ Export Reports to CSV

In [ ]:
# ── Export Reports to CSV ────────────────────────────────────────────

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
exports = []

# ── Performance Exports ──────────────────────────────────────────
if RUN_PERFORMANCE_DIAGNOSTICS:
    if 'df_cold_cache' in dir() and not df_cold_cache.empty:
        fname = f"perf_cold_cache_{timestamp}.csv"
        df_cold_cache.to_csv(fname, index=False)
        exports.append(("Cold Cache Queries", fname, len(df_cold_cache)))

    if 'df_long_running' in dir() and not df_long_running.empty:
        fname = f"perf_long_running_queries_{timestamp}.csv"
        df_long_running.to_csv(fname, index=False)
        exports.append(("Long-Running Queries", fname, len(df_long_running)))

    if 'df_all_stats' in dir() and not df_all_stats.empty:
        fname = f"perf_statistics_freshness_{timestamp}.csv"
        df_all_stats.to_csv(fname, index=False)
        exports.append(("Statistics Freshness", fname, len(df_all_stats)))

    if 'df_schema_issues' in dir() and not df_schema_issues.empty:
        fname = f"perf_schema_issues_{timestamp}.csv"
        df_schema_issues.to_csv(fname, index=False)
        exports.append(("Schema Issues", fname, len(df_schema_issues)))

    if 'df_compaction' in dir() and not df_compaction.empty:
        fname = f"perf_compaction_health_{timestamp}.csv"
        df_compaction.to_csv(fname, index=False)
        exports.append(("Compaction Health", fname, len(df_compaction)))

# ── Soft Delete Export ───────────────────────────────────────────
if RUN_SOFT_DELETE_SCAN and not df_deleted.empty:
    fname = f"soft_deleted_files_{timestamp}.csv"
    df_deleted.to_csv(fname, index=False)
    exports.append(("Soft-Deleted Files", fname, len(df_deleted)))

# ── Summary ──────────────────────────────────────────────────────
if exports:
    print("✅ Reports exported:")
    for name, fname, rows in exports:
        print(f"   📄 {name}: {fname} ({rows} rows)")
else:
    print("ℹ️  No data to export.")

---

## 📝 Notes

### Performance Checks Reference

| Check | DMV / API | What It Detects |
|-------|-----------|-----------------|
| Cold Cache | `queryinsights.exec_requests_history` | Queries loading data from OneLake instead of cache |
| Long-Running | `queryinsights.long_running_queries` | Queries with high median/max execution time |
| Frequently Run | `queryinsights.frequently_run_queries` | Most-executed queries (optimization candidates) |
| Statistics | `sys.stats` + `dm_db_stats_properties` | Stale statistics needing update |
| Schema | `sys.columns` + `sys.types` | Oversized strings, inefficient data types |
| Locks | `sys.dm_tran_locks` | Active locks and blocking |
| Compaction | `sys.dm_db_partition_stats` | Small/fragmented row groups |
| V-Order | Fabric REST API | V-Order and maintenance configuration |

### OneLake Soft Delete Key Facts

| Fact | Detail |
|------|--------|
| Retention Period | 7 days (fixed, not configurable) |
| Billing | Soft-deleted data billed at same rate as active data |
| Recovery Methods | Azure Storage Explorer, PowerShell (`Az.Storage`), REST API |
| Access Required | Write access to the item |
| Scope | All OneLake files (Lakehouse, Warehouse) |

### Best Practices Summary (from Microsoft Guidelines)

1. **Statistics**: Run `UPDATE STATISTICS` during maintenance windows after bulk data changes
2. **String columns**: Use specific `varchar(n)` lengths, avoid `varchar(8000)` or `varchar(max)`
3. **Data types**: Use `int`/`bigint` instead of `decimal(18,0)` for whole numbers
4. **Transactions**: Keep short-lived, always COMMIT/ROLLBACK, add retry logic
5. **Ingestion**: Batch writes, use `COPY INTO` with 100 MB–1 GB files, avoid trickle inserts
6. **Compaction**: Warehouse auto-compacts; for Lakehouse run `OPTIMIZE` after significant changes
7. **V-Order**: Keep enabled for read-heavy workloads; only disable for write-heavy staging
8. **Data clustering**: Cluster columns used in WHERE/JOIN predicates for file skipping
9. **Nullable columns**: Use `NOT NULL` when the data model allows
10. **Region alignment**: Keep data in the same region as your Fabric capacity

### Permissions Required
- **Workspace Admin** or **Member** role for full access
- **Write access** to items for restoring soft-deleted files